# **Lidar cone detection**

## **Setup**

Setup the repo and the model.

### **Clone the repositories and import the libraries**

Clone the repository from the smart application course organization and the one for the yolov5 model and import the libraries needed to work with the porject.

In [ ]:
!git clone --recurse-submodules "https://github.com/unipi-smartapp-2021/sensory-cone-detection"   # clone our repository
!git clone "https://github.com/ultralytics/yolov5"  # clone the model repository

Cloning into 'lidar-cone-detection'...
remote: Enumerating objects: 2899, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (33/33), done.
remote: Total 2899 (delta 15), reused 0 (delta 0), pack-reused 2865
Receiving objects: 100% (2899/2899), 121.76 MiB | 33.29 MiB/s, done.
Resolving deltas: 100% (515/515), done.
Submodule 'sensory/scripts/yolov5' (https://github.com/ultralytics/yolov5) registered for path 'sensory/scripts/yolov5'
Submodule 'src/lidar/ros_numpy' (https://github.com/eric-wieser/ros_numpy) registered for path 'src/lidar/ros_numpy'
Cloning into '/content/lidar-cone-detection/sensory/scripts/yolov5'...
remote: Enumerating objects: 10485, done.        
remote: Counting objects: 100% (2/2), done.        
remote: Compressing objects: 100% (2/2), done.        
remote: Total 10485 (delta 0), reused 1 (delta 0), pack-reused 10483        
Receiving objects: 100% (10485/10485), 10.65 MiB | 25.06 MiB/s, done.
Resolving deltas: 100% (7250/7250), 

The pypcd package has an error, fix it.

In [ ]:
!sed -i "s/import cStringIO as sio/from io import StringIO as sio/g" ../usr/local/lib/python3.7/dist-packages/pypcd/pypcd.py

Import the libraries and mount the drive.

In [ ]:
from tqdm import tqdm
import zipfile
import tarfile
from google.colab import drive
drive.mount('/content/drive')  # mount the drive

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### **Manage the dataset**

If the dataset is inside a rosbag extract it, otherwise manage it with the manage_dataset.py method.

In [ ]:
with zipfile.ZipFile("/content/drive/Shareddrives/Sensory_data/medium_ds2.zip", mode='r') as z:
        z.extractall()

Manage the dataset by following what is suggested in https://github.com/ultralytics/yolov5/wiki/Train-Custom-Data chapter 1.3. Keep in mind to modify the directory from which you're moving the images (ConeDataset for the first dataset or TRset for the full one).

You can decide to convert the images to grayscale ones or not, then the dataset is splited into train, validation and test sets.

In [ ]:
!python sensory-cone-detection/src/manage_dataset.py --dataset medium_ds --path sensory-cone-detection/src/datasets/dataset_lidar

### **Check the yolov5 model and import the weights**

Check if yolov5 is ready and fine.

In [ ]:
%cd yolov5
from yolov5.utils import notebook_init
display = notebook_init()  # checks
%cd ../

YOLOv5 🚀 v6.0-124-g1075488 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)


Setup complete ✅
/content


Install yolov5 requirements.

In [ ]:
!pip install -r yolov5/requirements.txt

Import the yolov5 model, all the available models can be found here: https://github.com/ultralytics/yolov5/releases/.

In [ ]:
import requests
url = 'https://github.com/ultralytics/yolov5/releases/download/v6.0/yolov5s.pt'
r = requests.get(url, allow_redirects=True)
open('yolov5s.pt', 'wb').write(r.content)

14698491

## **Convert from rosbag to pcd**

If the dataset is inside a rosbag convert each message to a pcd file by calling the right method.

In [ ]:
!python sensory-cone-detection/src/lidar/convert_rosbag_to_pcd.py --rosbag /content/lidar_dataset/2021-11-28-16-29-25.bag --path sensory-cone-detection/src/lidar/pcd_lidar

  0% 0/1800 [00:00<?, ?it/s]95961901796
  0% 0/1800 [00:00<?, ?it/s]


Save the pcd files inside a tar.gz by using the next command.

In [ ]:
!python -m tarfile -c lidar_dataset.tar.gz "lidar-cone-detection/src/Lidar/pcd_outputs/"

## **Train the model**

Train the model following the guide from https://github.com/ultralytics/yolov5/blob/master/tutorial.ipynb.

**Main arguments:**
*   *--img*, image size.
*   *--batch*, batch size.
*   *--epochs*, number of epochs.
*   *--data*, the yaml file that specifies where datasets are.
*   *--weights*, the model weights (you can find them here https://github.com/ultralytics/yolov5/releases).
*   *--cache*, to save a cache file of the train and validation sets.
*   *--project*, where to save the results.

**An example of train call would be:**

!python yolov5/train.py --img img_size --batch batch_size --epochs epochs --data dataset.yaml --weights model --cache --project runs/train








In [ ]:
!python yolov5/train.py --img 480 --batch 16 --epochs 100 --data sensory-cone-detection/src/lidar/conedataset_lidar.yaml --weights yolov5s.pt --cache --project train

train: weights=/content/yolov5s.pt, cfg=, data=/content/lidar-cone-detection/src/lidar/conedataset_lidar.yaml, hyp=yolov5/data/hyps/hyp.scratch.yaml, epochs=100, batch_size=16, imgsz=480, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=ram, image_weights=False, device=, multi_scale=False, single_cls=False, adam=False, sync_bn=False, workers=8, project=train, name=exp, exist_ok=False, quad=False, linear_lr=False, label_smoothing=0.0, patience=100, freeze=0, save_period=-1, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=-1, artifact_alias=latest
github: up to date with https://github.com/ultralytics/yolov5 ✅
YOLOv5 🚀 v6.0-124-g1075488 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

hyperparameters: lr0=0.01, lrf=0.1, momentum=0.937, weight_decay=0.0005, warmup_epochs=3.0, warmup_momentum=0.8, warmup_bias_lr=0.1, box=0.05, cls=0.5, cls_pw=1.0, obj=1.0, obj_pw=1.0, iou_t=0.2, anchor_t=4.0, fl_gamma=0.0, hsv_h=0.015, hsv

## **Evaluate the model**

Use val.py if you have a test set defined in conedataset.yaml. Remember to use --task test or the model will work on the validation set.

In [ ]:
!python yolov5/val.py --weights drive/Shareddrives/Sensory_data/lidar_weights.pt --img 480 --conf 0.5 --data sensory-cone-detection/src/lidar/conedataset_lidar.yaml --task test --project test

## **Inference**

Use detect.py if you have no labeled images or you want to try the model on your custom data.

In [ ]:
import time
import numpy as np
start = np.floor(time.time()*1000)
!rm -rf lidar_inference/exp
print(np.floor(time.time()*1000-start))
!python yolov5/detect.py --weights /content/drive/Shareddrives/Sensory_data/lidar_weights.pt --img 480 --conf 0.5 --source sensory-cone-detection/src/datasets/dataset_lidar/images/ --save-txt --project lidar_inference --hide-labels --line-thickness 2